In [1]:
# Basic Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
# Keras Libs
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout
# Sklearn Libs
from sklearn.metrics import mean_squared_error
from bcb_service import get_currency

In [2]:
def create_timeseries(series, ts_lag =1):
  dataX = []
  dataY = []

  n_rows = len(series) - ts_lag

  for i in range(n_rows-1):
    a = series[i:(i + ts_lag)]
    dataX.append(a)
    dataY.append(series[i + ts_lag])

  X, Y = np.array(dataX), np.array(dataY)
  return X, Y

In [43]:
data = get_currency(60)
print(data["D"])
data = data["USD"]
size_data = len(data)
size_data

               USD
Date              
2023-10-23  5.0164
2023-10-24  5.0065
2023-10-25  4.9981
2023-10-26  5.0055
2023-10-27  4.9480
2023-10-30  5.0074
2023-10-31  5.0575
2023-11-01  5.0194
2023-11-03  4.8910
2023-11-06  4.8999
2023-11-07  4.8670
2023-11-08  4.8855
2023-11-09  4.9007
2023-11-10  4.9219
2023-11-13  4.9247
2023-11-14  4.8682
2023-11-16  4.8575
2023-11-17  4.8849
2023-11-20  4.8723
2023-11-21  4.8806
2023-11-22  4.8968
2023-11-23  4.8931
2023-11-24  4.8927
2023-11-27  4.8951
2023-11-28  4.8867
2023-11-29  4.8933
2023-11-30  4.9355
2023-12-01  4.9191
2023-12-04  4.9091
2023-12-05  4.9522
2023-12-06  4.9031
2023-12-07  4.8949
2023-12-08  4.9158
2023-12-11  4.9440
2023-12-12  4.9476
2023-12-13  4.9580
2023-12-14  4.8912
2023-12-15  4.9397
2023-12-18  4.9399
2023-12-19  4.8663


40

In [4]:
train, test = data[0:size_data//2], data[size_data//2:size_data]

In [5]:

ts_lag = 3
trainX, trainY = create_timeseries(train, ts_lag)
testX, testY = create_timeseries(test, ts_lag)

# reshape input data to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

C:\Users\Lucas\AppData\Local\Temp\ipykernel_1664\2863392632.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dataY.append(series[i + ts_lag])


In [6]:
model = Sequential()
model.add(LSTM(200, input_shape = trainX[0].shape))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

# Compile the model
model.compile(loss = "mean_absolute_error", optimizer = "adamax")

# Fit the model
model.fit(trainX, trainY, epochs = 500, batch_size = 2, verbose =0)

In [7]:

train_predict = model.predict(trainX)
test_predict =model.predict(testX)

1/1 [==============================] - 0s 31ms/step


In [9]:
rmse = np.sqrt(mean_squared_error(test_predict, testY))
print("O RMSE do testX é: {:.2f}".format(rmse))

O RMSE do testX é: 0.09


In [42]:
# print(testX)
result = model.predict([[[data[-3], data[-2], data[-1]]]])
result[0][0]

1/1 [==============================] - 0s 53ms/step


C:\Users\Lucas\AppData\Local\Temp\ipykernel_1664\1580840837.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  result = model.predict([[[data[-3], data[-2], data[-1]]]])


5.0016